## ***Explore RunnableLambda***
* use runnableLambda you can do convert your custom function can be work as Runnable

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain.schema.runnable import RunnablePassthrough, RunnableSequence, RunnableParallel, RunnableLambda

groq_api_key=os.getenv("GROQ_API_KEY")

model= ChatGroq(groq_api_key=groq_api_key,model_name="qwen-2.5-32b")
model2= ChatGroq(groq_api_key=groq_api_key,model_name="llama3-8b-8192")


## How work Runnable Lambda

In [9]:
## Create a py function
def wordCounter(text:str):
    return len(text.split())

## make Runnable Lambda
runnable_word_counter =RunnableLambda(wordCounter)

In [10]:
runnable_word_counter.invoke("Hello sir how are you?")

5

In [11]:
prompt = PromptTemplate(
    template="Write a joke about this {topic}",
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template="Explain the following joke {text}",
    input_variables=['text']
)

parser = StrOutputParser()
joke_gen_chain = RunnableSequence(prompt, model, parser)

In [15]:
parallel_chain = RunnableParallel(
    {
        'joke': RunnablePassthrough(),
        'word_count': runnable_word_counter
    }
)

In [16]:
final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

In [17]:
final_chain.get_graph().print_ascii()

              +-------------+              
              | PromptInput |              
              +-------------+              
                     *                     
                     *                     
                     *                     
            +----------------+             
            | PromptTemplate |             
            +----------------+             
                     *                     
                     *                     
                     *                     
               +----------+                
               | ChatGroq |                
               +----------+                
                     *                     
                     *                     
                     *                     
            +-----------------+            
            | StrOutputParser |            
            +-----------------+            
                     *                     
                     *          

In [18]:
response = final_chain.invoke({
    'topic': 'Laptop'
})

In [19]:
response['joke']

'Why did the laptop go to the doctor?\n\nBecause it had a virus and it felt a bit blue!'

In [20]:
response['word_count']

19